# HW_03 基于HMM与的标注分词
## 1) 单字处理
将199801.txt处理成单字与4个构词位置标记的分词语料。 4个构词位置标记为：

    B ：词首，M ：词中，E：词尾，S：单独成词。

输入：迈向/v 充满/v 希望/n 的/u 新/a 世纪/n

输出：迈/B 向/E 充/B 满/E 希/B 望/E 的/S 新/S 世/B 纪/E

In [2]:
import re

def neighborhood(iterable):
    iterator = iter(iterable)
    prev_item = None
    current_item = next(iterator)  # throws StopIteration if empty.
    for next_item in iterator:
        yield (prev_item, current_item, next_item)
        prev_item = current_item
        current_item = next_item
    yield (prev_item, current_item, None)

'''
# target 
    /B begin
    /M middle
    /E end
    /S single
'''

text = open("/home/tcj_wsl/ML-SXU/hw_03/199801.txt", mode='r',encoding="gbk")
resultStr = open('/home/tcj_wsl/ML-SXU/hw_03/dicStr.txt', mode='w')

while True:
    lineCache = text.readline()
    lineCache.encode('utf-8')
    
    if not lineCache:
        break

    newLine = ""
    
    lineCache = re.sub("/[A-z]+", "", lineCache)
    lineCache = re.sub("\[", "", lineCache)
    lineCache = re.sub("\]", "", lineCache)

    for i, j, k in neighborhood(lineCache):
        if i == None:
            newLine += j
        elif k == None:
            newLine += '\n'
        elif (i == " ") \
                    and ('\u4e00' <= j <= '\u9fa5' or '\uff10' <= j <= '\uff19') \
                    and ('\u4e00' <= k <= '\u9fa5' or '\uff10' <= k <= '\uff19') :
            newLine += j+"\B"
        elif ('\u4e00' <= i <= '\u9fa5' or '\uff10' <= i <= '\uff19') \
                    and ('\u4e00' <= j <= '\u9fa5' or '\uff10' <= j <= '\uff19') \
                    and ('\u4e00' <= k <= '\u9fa5' or '\uff10' <= k <= '\uff19'):
            newLine += j+"\M"
        elif ('\u4e00' <= i <= '\u9fa5' or '\uff10' <= i <= '\uff19') \
                    and ('\u4e00' <= j <= '\u9fa5' or '\uff10' <= j <= '\uff19') \
                    and (k == ' '):
            newLine += j+"\E"
        elif (i == " ") \
                    and ('\u4e00' <= j <= '\u9fa5' or '\uff10' <= j <= '\uff19') \
                    and (k == ' '):
            newLine += j + "\S"
        else:
            newLine += j
    
    # print(newLine)
    resultStr.write(newLine)


## 2) 语料分割与基础统计
将语料分成测试集与训练集（一般为4：1的比例）。在训练集上统计估算初始概率、发射概率、转移概率所需的参数。

In [1]:

text = open("/home/tcj_wsl/ML-SXU/hw_03/dicStr.txt", mode='r')
resultTrain = open("/home/tcj_wsl/ML-SXU/hw_03/retTrain.txt", mode='w')
resultValidation = open("/home/tcj_wsl/ML-SXU/hw_03/retValidation.txt", mode='w')

lineNum = 0
while True:    
    lineCache = text.readline()
    
    if not lineCache:
        break

    if (lineNum % 5) == 0:
        resultValidation.write( lineCache )
    else: 
        resultTrain.write( lineCache )

    lineNum += 1

resultTrain.close()
resultValidation.close()
text.close()

$$初始概率 = \frac{该字作为句首的次数}{句子数}$$
因此需要统计有：  
* 句子数
* 标记$t_1$作为句首的次数

$$发射概率 = \frac{Num(w_i,t_i)}{Num(t_i)}$$
因此需要统计有：  
* 不同$w_i|t_i$的出现次数
* 不同$t_i$的次数

$$转移概率 = \frac{t_{i-1}变为t_i的次数}{t_{i-1}词性个数} $$
* 词性$t_{i-1}变为t_i$的次数

$t:状态，这里是词性（词标注）或字标注\\ w: word词、字段$



In [30]:
#!/usr/bin/python3

import re

# -----------------------------词统计

def inputDict(inString, dict):
    if inString in dict:
        dict[inString] += 1
    else:
        dict[inString] = 1

text = open("/home/tcj_wsl/ML-SXU/hw_03/retTrain.txt", mode='r')

propertyDict = {}
'''不同$t_i$的次数'''
wordsDict = {}      
'''不同$$w_i|t_i$$的出现次数'''
firstDict = {}      
'''标记$t_1$作为句首的次数'''
transDict = {}
'''词性$t_{i-1}变为t_i$的次数'''
strNum = 0          
'''句子数'''

while True:
    lineCache = text.readline()
    if not lineCache:
        break
        
    pattern = re.compile(r'.\\[B,M,E,S]')   #字\标注
    pattern2 = re.compile(r"\\[B,M,E,S]")   #标注
    words = pattern.findall(lineCache)
    propertys = pattern2.findall(lineCache)
    # print(lineCache)
    # print(words)
    # print(propertys)

    if len(propertys) == 0:
        continue
    strNum += 1
    
    # print(propertys[0],strNum)

    inputDict(propertys[0], firstDict)
    for i in words:
        inputDict(i, wordsDict)
    for i in propertys:
        # print(i)
        inputDict(i, propertyDict)
    for i in range(len(propertys)-1):
        # print()
        inputDict(propertys[i]+propertys[i+1], transDict)
        # if (propertys[i]+propertys[i+1] == "\B\B"): #来自于１\B８．７％  ，  预\B计\E  全\
        #     print(lineCache)

    # break

# print(propertyDict)
# print(wordsDict)
# print(firstDict)
# print(strNum)

# trans dict 后处理
remove = []
for i in transDict:
    if transDict[i] < 1000:
        remove.append(i)
for i in remove:
    transDict.pop(i)

print(transDict)


{'\\B\\M': 65794, '\\M\\M': 28338, '\\M\\E': 65844, '\\E\\B': 284751, '\\B\\E': 403558, '\\E\\S': 172466, '\\S\\B': 174525, '\\S\\S': 106319}


## 3) Viterbi字标注与检验
利用Viterbi算法，实现基于HMM的字标注的分词程序

在训练集上完成了数据的统计，在测试集上进行预测给定字的标注